# Epoch Single File
Added ReadyToSleep phases etc.  Has been copied into main multi pipeline.

In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import run_yasa
import logging
import mne
import yasa
import os
import argparse
import os

import mne
import numpy as np
import pandas as pd
from brainflow.board_shim import BoardShim, BoardIds
from brainflow.data_filter import DataFilter
log = lambda msg: logging.info(msg)


# Load data

In [ ]:
input_dir = "C:\\dev\\play\\brainwave-data"
stats_df = pd.read_csv(input_dir + os.path.sep + "stats.csv")


In [ ]:
from sleep_events import load_days_data

days_data = load_days_data()
days_data_orig = days_data.copy()

In [ ]:
from sleep_events import pimp_my_days_data

days_data = pimp_my_days_data(days_data)

In [ ]:
days_data['asleepTime'].info()

In [ ]:
[col for col in days_data.columns if "lep:" in col.lower()]

In [ ]:
# yasa_df

In [47]:
from sleep_events import convert_timestamp_to_uk


def process_file(dir_name, input_file):
    day_and_night_of_date = datetime.strptime(dir_name, "%Y-%m-%d-%H-%M-%S")
    day_and_night_of = day_and_night_of_date.date().isoformat()
    yasa_df = run_feature_pipeline.cached_pipeline(log, input_file, stats_df)
    yasa_df['dayAndNightOf'] = day_and_night_of
    row = days_data[days_data['dayAndNightOf'] == day_and_night_of]
    if not row.empty:
        yasa_df['asleepTime'] = row['asleepTime'].iloc[0]
        yasa_df['wakeTime'] = row['wakeTime'].iloc[0]
        yasa_df['gotIntoBedTime'] = row['gotIntoBedTime'].iloc[0]
        yasa_df['readyToSleepTime'] = row['readyToSleepTime'].iloc[0]
        yasa_df['LEPTime'] = row['LEPTime'].iloc[0]
        # print(row['asleepTime'].iloc[0])
        # print(row['asleepTime'].iloc[0].dtype)
    
        # print(row['asleepTime'].dtype)
        # # print(row['asleepTime'].values[0])
        # # print(row['asleepTime'].values[0].dtype)
        # print(row['asleepTime'].iloc[0])
        # print(row['asleepTime'].iloc[0].dtype)
        # print(yasa_df['asleepTime'].dtype)

    yasa_df['Timestamp'] = df['Timestamp'].apply(convert_timestamp_to_uk)
    return yasa_df
    
df = process_file("2024-11-01-21-32-22", "C:\\dev\\play\\brainwave-data\\2024-11-01-21-32-22\\raw.fif")

In [48]:
df[['Stage', 'Timestamp', 'asleepTime', 'dayAndNightOf']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1026 entries, 0 to 1025
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype                        
---  ------         --------------  -----                        
 0   Stage          1026 non-null   object                       
 1   Timestamp      1026 non-null   datetime64[ns, Europe/London]
 2   asleepTime     1026 non-null   datetime64[ns, Europe/London]
 3   dayAndNightOf  1026 non-null   object                       
dtypes: datetime64[ns, Europe/London](2), object(2)
memory usage: 40.1+ KB


In [49]:
from sleep_events import convert_timestamps_to_uk
from datetime import datetime
import os
import run_feature_pipeline
errors = []

dfs = []
# Could get these working later
skip_list = ['2024-07-23-22-40-25', '2024-07-28-22-29-49', '2024-09-18-21-25-08', '2024-09-18-21-28-11', '2024-09-19-21-29-42']

def assign_timestamps_with_tz(df, source_col, target_col):
    """
    Assign timezone-aware timestamps from one column to another in a DataFrame.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame to assign the values to
    source_col : str
        The name of the column containing the source timestamps
    target_col : str
        The name of the column to assign the values to
    """
    # Check if the target column exists
    if target_col not in df.columns:
        df[target_col] = pd.NaT

    # Assign the values while preserving the timezone
    df[target_col] = df[source_col]


for root, dirs, files in os.walk(input_dir):
    for idx, dir_name in enumerate(dirs):
        input_file = os.path.join(root, dir_name, "raw.fif")
        if dir_name in skip_list:
            log(f"Skipping {idx} of {len(dirs)}: " + input_file)
            continue
        try:
            log("Processing file: " + input_file)
            if os.path.exists(input_file):
                yasa_df = process_file(dir_name, input_file)
                dfs.append(yasa_df)

        except Exception as e:
            log("Error processing file: " + input_file)
            errors.append("Error processing file: " + input_file + " - " + str(e))
            log(e)

for err in errors:
    log(err)
    
yasa_df = pd.concat(dfs)
yasa_df_orig = yasa_df.copy()

C:\dev\play\brainwave-processor\sleep_events.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ha_events_for_file = ha_events[ha_events['timestamp'] >= start_date][ha_events['timestamp'] <= end_date][ha_events['event'] == 'wake']
C:\dev\play\brainwave-processor\sleep_events.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ha_events_for_file = ha_events[ha_events['timestamp'] >= start_date][ha_events['timestamp'] <= end_date][ha_events['event'] == 'wake']


In [50]:
row = days_data[days_data['dayAndNightOf'] == '2024-11-01']
# yasa_df['asleepTime'] = row['asleepTime'].iloc[0]
# yasa_df['wakeTime'] = row['wakeTime'].iloc[0]
# yasa_df['gotIntoBedTime'] = row['gotIntoBedTime'].iloc[0]
# yasa_df['readyToSleepTime'] = row['readyToSleepTime'].iloc[0]
# yasa_df['LEPTime'] = row['LEPTime'].iloc[0]
# print(row['asleepTime'].iloc[0])
# print(row['asleepTime'].iloc[0].dtype)
print(row)
print(row.empty)
print(row['asleepTime'].dtype)
print(row['asleepTime'].values[0])
print(row['asleepTime'].values[0].dtype)
print(row['asleepTime'].iloc[0])
print(row['asleepTime'].iloc[0].dtype)
print(yasa_df['asleepTime'].dtype)


    dayAndNightOf  exercise:fitbit:heartRateZones:OutofRange:caloriesOut  \
194    2024-11-01                                          962.97656       

     exercise:fitbit:activityLevels:very:distance  \
194                                        0.3532   

     exercise:byExercise:Run:averageHeartRate  \
194                                       0.0   

     exercise:fitbit:heartRateZones:Peak:min  \
194                                    160.0   

     drugsAndSupplements:MultivitQuantity  \
194                                   0.0   

     exercise:byExercise:Treadmill:firstDurationSecs  \
194                                              0.0   

     exercise:byExercise:Treadmill:totalSteps  exercise:fitbit:floors  \
194                                       0.0                     5.0   

     date:month  ...  drugsAndSupplements:VitaminD3FirstSSM:vsDayMinus1  \
194          11  ...                                                NaN   

     night:morningQuestionnaire:pajamas:bo

AttributeError: 'Timestamp' object has no attribute 'dtype'

In [34]:
errors

['Error processing file: C:\\dev\\play\\brainwave-data\\2024-10-02-21-20-34\\raw.fif - Sliding window size may not exceed size of selected axis',
 'Error processing file: C:\\dev\\play\\brainwave-data\\2024-11-05-21-34-15\\raw.fif - No file or directory found at ./microwakings_multi1.h5']

In [51]:
yasa_df[['Stage', 'Timestamp', 'asleepTime']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 68930 entries, 0 to 1244
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype                        
---  ------      --------------  -----                        
 0   Stage       68930 non-null  object                       
 1   Timestamp   61691 non-null  datetime64[ns, Europe/London]
 2   asleepTime  64200 non-null  object                       
dtypes: datetime64[ns, Europe/London](1), object(2)
memory usage: 2.1+ MB


In [ ]:
yasa_df[['Stage', 'Timestamp', 'asleepTime']].info()

In [ ]:
yasa_df['Timestamp'].info()

In [ ]:
yasa_df['Timestamp'].to_datetime()

In [ ]:
yasa_df['Timestamp2'] = pd.to_datetime(yasa_df['Timestamp'].astype(str), format='mixed', errors='coerce')

In [ ]:
yasa_df['Timestamp2'].isna().any()

In [52]:
yasa_df['asleepTime'].apply(type).value_counts()

asleepTime
<class 'pandas._libs.tslibs.timestamps.Timestamp'>    64200
<class 'pandas._libs.tslibs.nattype.NaTType'>          4730
Name: count, dtype: int64

In [ ]:
yasa_df['Timestamp3'] = pd.to_datetime(yasa_df['Timestamp2'], errors='coerce')
print(yasa_df['Timestamp3'].dtype)


In [ ]:


yasa_df = convert_timestamps_to_uk(yasa_df)

In [ ]:
#df_timestamps[['TimestampUK', 'Timestamp']]

In [ ]:
# yasa_df['TimestampUTC'] = yasa_df['Timestamp3'].tz_localize('UTC')

In [ ]:
# yasa_df[yasa_df['Timestamp2'].isna()]['Timestamp']

In [ ]:
# yasa_df['Timestamp2'].info()

In [ ]:
# print(yasa_df['Timestamp'].value_counts().head())


In [ ]:
# print(yasa_df['Timestamp'].head())


In [ ]:
# Attempt conversion with debug information
# problematic_rows = []
# for idx, timestamp in yasa_df['Timestamp'].items():
#     try:
#         pd.to_datetime(timestamp, format='ISO8601')
#     except Exception as e:
#         problematic_rows.append((idx, timestamp, str(e)))
#         if len(problematic_rows) >= 5:  # Show first 5 problems
#             break
# 
# if problematic_rows:
#     print("First few problematic timestamps:")
#     for idx, ts, error in problematic_rows:
#         print(f"Index {idx}: {ts}")
#         print(f"Error: {error}\n")

In [ ]:
# # Show the actual format of the first few timestamps
# print("Sample timestamp format:")
# print(yasa_df['Timestamp'].head().apply(lambda x: f"Length: {len(str(x))}, Value: {x}"))

In [ ]:
# # Ensure the 'Timestamp' column is in datetime format
# yasa_df['TimestampISO'] = pd.to_datetime(yasa_df['Timestamp'], format='ISO8601', errors='coerce')
# 
# # Localize the 'Timestamp' column to UTC
# yasa_df['TimestampUTC'] = yasa_df['TimestampISO'].tz_localize('UTC')
# 
# # Convert the 'Timestamp' column from UTC to UK time
# yasa_df['TimestampUK'] = yasa_df['TimestampUTC'].dt.tz_convert('Europe/London')

In [ ]:
# yasa_df['TimestampISO'].isna().any()

In [ ]:
# print(yasa_df['TimestampISO'].dtype)


In [ ]:
# yasa_df[['Stage', 'Timestamp', 'TimestampISO', 'asleepTime']]

In [ ]:
# # Ensure the 'Timestamp' column is in datetime format
# yasa_df['TimestampISO'] = pd.to_datetime(yasa_df['Timestamp'], format='ISO8601')
# 
# # Convert the 'Timestamp' column from UTC to UK time
# yasa_df['TimestampUK'] = yasa_df['Timestamp'].dt.tz_convert('Europe/London')

In [66]:
yasa_df['timeSinceSleep'] = yasa_df['Timestamp'] - yasa_df['asleepTime']
yasa_df['timeSinceReadyToSleep'] = yasa_df['Timestamp'] - yasa_df['readyToSleepTime']
yasa_df['timeSinceGotIntoBed'] = yasa_df['Timestamp'] - yasa_df['gotIntoBedTime']
yasa_df['timeSinceLEP'] = yasa_df['Timestamp'] - yasa_df['LEPTime']
#yasa_df['timeUntilWake'] = yasa_df['wakeTime'] - yasa_df['Timestamp']
yasa_df['15MinsBeforeSleep'] = ((yasa_df['timeSinceSleep'] >= pd.Timedelta(minutes=-15)) &
                                (yasa_df['timeSinceSleep'] <= pd.Timedelta(minutes=0))).astype(int)
yasa_df['30MinsBeforeSleep'] = ((yasa_df['timeSinceSleep'] >= pd.Timedelta(minutes=-30)) &
                                (yasa_df['timeSinceSleep'] <= pd.Timedelta(minutes=0))).astype(int)
yasa_df['45MinsBeforeSleep'] = ((yasa_df['timeSinceSleep'] >= pd.Timedelta(minutes=-45)) &
                                (yasa_df['timeSinceSleep'] <= pd.Timedelta(minutes=0))).astype(int)
yasa_df['60MinsBeforeSleep'] = ((yasa_df['timeSinceSleep'] >= pd.Timedelta(minutes=-60)) &
                                (yasa_df['timeSinceSleep'] <= pd.Timedelta(minutes=0))).astype(int)
yasa_df['DuringReadyToSleep'] = ((yasa_df['Timestamp'] >= yasa_df['readyToSleepTime']) & (yasa_df['Timestamp'] <= yasa_df['asleepTime'])).astype(int)
yasa_df['SleepHour1'] = ((yasa_df['timeSinceSleep'] <= pd.Timedelta(hours=1)) & (yasa_df['timeSinceSleep'] >= pd.Timedelta(hours=0))).astype(int)
yasa_df['SleepHour2'] = ((yasa_df['timeSinceSleep'] <= pd.Timedelta(hours=2)) & (yasa_df['timeSinceSleep'] >= pd.Timedelta(hours=1))).astype(int)
yasa_df['SleepHour3'] = ((yasa_df['timeSinceSleep'] <= pd.Timedelta(hours=3)) & (yasa_df['timeSinceSleep'] >= pd.Timedelta(hours=2))).astype(int)
yasa_df['SleepHour4'] = ((yasa_df['timeSinceSleep'] <= pd.Timedelta(hours=4)) & (yasa_df['timeSinceSleep'] >= pd.Timedelta(hours=3))).astype(int)
yasa_df['SleepHour5'] = ((yasa_df['timeSinceSleep'] <= pd.Timedelta(hours=5)) & (yasa_df['timeSinceSleep'] >= pd.Timedelta(hours=4))).astype(int)
yasa_df['SleepHour6'] = ((yasa_df['timeSinceSleep'] <= pd.Timedelta(hours=6)) & (yasa_df['timeSinceSleep'] >= pd.Timedelta(hours=5))).astype(int)
yasa_df['SleepHour7'] = ((yasa_df['timeSinceSleep'] <= pd.Timedelta(hours=7)) & (yasa_df['timeSinceSleep'] >= pd.Timedelta(hours=6))).astype(int)
yasa_df['SleepHour8'] = ((yasa_df['timeSinceSleep'] <= pd.Timedelta(hours=8)) & (yasa_df['timeSinceSleep'] >= pd.Timedelta(hours=7))).astype(int)


C:\Users\graha\AppData\Local\Temp\ipykernel_34116\1972786365.py:1: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  yasa_df['timeSinceSleep'] = yasa_df['Timestamp'] - yasa_df['asleepTime']
C:\Users\graha\AppData\Local\Temp\ipykernel_34116\1972786365.py:2: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  yasa_df['timeSinceReadyToSleep'] = yasa_df['Timestamp'] - yasa_df['readyToSleepTime']
C:\Users\graha\AppData\Local\Temp\ipykernel_34116\1972786365.py:3: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  yasa_df['timeSinceGotIntoBed'] = yasa_df['Timestamp'] - yasa_df['gotIntoBedTime']
C:\Users\graha\AppData\Local\Temp\ipykernel_34116\1972786365.py:4: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  yasa_df['timeSinceLEP'] = yasa_df['Timestamp'] - yasa_df['LEPTime']
C:\Users\graha\AppData\Local\Temp\ipykernel_34116\1

In [68]:
yasa_df[yasa_df['dayAndNightOf'] == '2024-11-01'][['Stage', 'dayAndNightOf', 'Timestamp', 'readyToSleepTime', 'asleepTime', 'DuringReadyToSleep', '60MinsBeforeSleep', '30MinsBeforeSleep', 'SleepHour1', 'SleepHour2']]

,Stage,dayAndNightOf,Timestamp,readyToSleepTime,asleepTime,DuringReadyToSleep,60MinsBeforeSleep,30MinsBeforeSleep,SleepHour1,SleepHour2
epoch,,,,,,,,,,
0,W,2024-11-01,2024-11-01 21:32:22.923768044+00:00,2024-11-01 22:25:30+00:00,2024-11-01 22:31:23+00:00,0,1,0,0,0
1,W,2024-11-01,2024-11-01 21:32:52.923768044+00:00,2024-11-01 22:25:30+00:00,2024-11-01 22:31:23+00:00,0,1,0,0,0
2,W,2024-11-01,2024-11-01 21:33:22.923768044+00:00,2024-11-01 22:25:30+00:00,2024-11-01 22:31:23+00:00,0,1,0,0,0
3,W,2024-11-01,2024-11-01 21:33:52.923768044+00:00,2024-11-01 22:25:30+00:00,2024-11-01 22:31:23+00:00,0,1,0,0,0
4,W,2024-11-01,2024-11-01 21:34:22.923768044+00:00,2024-11-01 22:25:30+00:00,2024-11-01 22:31:23+00:00,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1021,W,2024-11-01,2024-11-02 06:02:52.923768044+00:00,2024-11-01 22:25:30+00:00,2024-11-01 22:31:23+00:00,0,0,0,0,0
1022,W,2024-11-01,2024-11-02 06:03:22.923768044+00:00,2024-11-01 22:25:30+00:00,2024-11-01 22:31:23+00:00,0,0,0,0,0
1023,W,2024-11-01,2024-11-02 06:03:52.923768044+00:00,2024-11-01 22:25:30+00:00,2024-11-01 22:31:23+00:00,0,0,0,0,0
